In [41]:
import imblearn
import numpy as np
import os
import pandas as pd
import scipy as sp
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import cross_val_score

In [4]:
PATH_TO_DATA = 'product-reviews-sentiment-analysis-light'
TRAIN = 'products_sentiment_train.tsv'
TEST = 'products_sentiment_test.tsv'

In [77]:
df_train = pd.read_csv(os.path.join(PATH_TO_DATA, TRAIN), sep='\t', header=None)
df_train.columns = ['text', 'lab']
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, TEST), sep='\t', index_col='Id')

In [15]:
df_train.columns = ['text', 'lab']
df_train.head()

,text,lab
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [86]:
df_train.lab.value_counts()

1    1274
0     726
Name: lab, dtype: int64

Ой, у нас количество классов не равно. Давайте потратим время на выравнивание количества классов, как правило, это дает улучшение предсказания (для классификатора, веторизатор при этим портится). Также при равенстве классов нам хватит accuracy для оценки модели. Выравнивать будет случайным дублированием отрицательных отзывов.
Оценивать будем по красс-валидации. При oversampling-e это не самый лучший вариант (так как могут получиться развиения, когда алгоритм видел отзыв заранее), поэтому найдем сначала "работающий достаточно хорошо" по необработанным данным.

In [22]:
X, y = df_train.text, df_train.lab

In [31]:
index_od_0s = X.index[y == 0]
X_1, y_1 = X[y == 1], y[y == 1]
index_oversample = np.random.choice(index_od_0s, size=df_train.lab.value_counts()[1])
X_resampled = np.concatenate([X[index_oversample],X_1])
y_resampled = np.concatenate([y[index_oversample],y_1])

In [36]:
y_resampled.shape[0]/2

1274.0

Вот теперь лучше. Давайте теперь проверим все векторизаторы и классификаторы со стандартными параметрами, а потом проведем поиск по сетке у лучших. 

In [81]:
for name_pre, preproc in [('Count', CountVectorizer), ('TFIDF', TfidfVectorizer), ('Hash', HashingVectorizer)]:
    for name_est, est in [('LR', LogisticRegression), ('SGD', SGDClassifier), ('SVM', LinearSVC)]:
        pipe = Pipeline([
            ('preproc', preproc()),
            ('est', est())
        ])
        acc = cross_val_score(pipe, X_resampled, y=y_resampled, scoring='accuracy', cv=5).mean()
        
        print('Pre: {}, Est: {}, \tacc: {}'.format(name_pre, name_est, acc))

Pre: Count, Est: LR, 	acc: 0.8590952601513047
Pre: Count, Est: SGD, 	acc: 0.8296788636714529
Pre: Count, Est: SVM, 	acc: 0.8630230044773816
Pre: TFIDF, Est: LR, 	acc: 0.8390721012814575
Pre: TFIDF, Est: SGD, 	acc: 0.8618403581905204
Pre: TFIDF, Est: SVM, 	acc: 0.8638042303535588
Pre: Hash, Est: LR, 	acc: 0.7998270804384746
Pre: Hash, Est: SGD, 	acc: 0.8394673459935156
Pre: Hash, Est: SVM, 	acc: 0.8563439864134631


In [80]:
# без ресампла
for name_pre, preproc in [('Count', CountVectorizer), ('TFIDF', TfidfVectorizer), ('Hash', HashingVectorizer)]:
    for name_est, est in [('LR', LogisticRegression), ('SGD', SGDClassifier), ('SVM', LinearSVC)]:
        pipe = Pipeline([
            ('preproc', preproc()),
            ('est', est())
        ])
        acc = cross_val_score(pipe, X, y=y, scoring='accuracy', cv=5).mean()
        
        print('Pre: {}, Est: {}, \tacc: {}'.format(name_pre, name_est, acc))

Pre: Count, Est: LR, 	acc: 0.7684956843480272
Pre: Count, Est: SGD, 	acc: 0.7334793373708586
Pre: Count, Est: SVM, 	acc: 0.754000653129082
Pre: TFIDF, Est: LR, 	acc: 0.7665031843949025
Pre: TFIDF, Est: SGD, 	acc: 0.7524918686991794
Pre: TFIDF, Est: SVM, 	acc: 0.7684856717854487
Pre: Hash, Est: LR, 	acc: 0.7490006281289258
Pre: Hash, Est: SGD, 	acc: 0.7564893936837105
Pre: Hash, Est: SVM, 	acc: 0.7689881780511127


HashingVectorizer, не имея вообще никаких обосновний для работы, почему-то показывает очень неплохой результат. Но он сравним с другими, а его нестабильность не позволяет верить в то, что мы получим устойчивое решение. 

Давайте оставим SVM с TFIDF.

In [92]:
vec = TfidfVectorizer()
vec.fit(X)

X_transformed = vec.transform(X)

est = LinearSVC()
est.fit(X_transformed, y)

X_test = vec.transform(df_test.text)

df_test['y'] = est.predict(X_test)
df_test[['y']].to_csv('sub_.csv')

Почему-то стабилизация вариантов только ухудшила результат. Выше - вариант, который получился лучшем на LeaderBoard. Так как он уже бьет бенчмарк, то докручивать что-то мне лень.